In [1]:
import requests


In [5]:
# retrieve download link
result = requests.post('https://europe-west3-lumpito.cloudfunctions.net/download_files', json={
    "company": "Alasco",
    "documents": ["gs", "si"]
})

In [6]:
result.text

'500 Internal Server Error: The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.'